In [ ]:
https://kkroening.github.io/ffmpeg-python/
https://github.com/kkroening/ffmpeg-python/tree/master/examples
https://kkroening.github.io/ffmpeg-python/


In [1]:
import glob, time
import ffmpeg, numpy as np

VIDEO_DIR = '../data/raw_data/video/'

files = glob.glob(VIDEO_DIR+'*')
vid_files = [ f for f in files if f.split('.')[-1].lower() in ['mp4', '3gp', 'avi'] ]
vid_files

['../data/raw_data/video/594371324.545536.mp4',
 '../data/raw_data/video/594371324.037443.mp4',
 '../data/raw_data/video/594371324.202440.mp4',
 '../data/raw_data/video/594371324.732749.mp4',
 '../data/raw_data/video/594371324.375344.mp4',
 '../data/raw_data/video/594371324.126019.mp4',
 '../data/raw_data/video/594371323.935324.mp4',
 '../data/raw_data/video/594371324.458069.mp4',
 '../data/raw_data/video/594371324.293383.mp4']

In [2]:
input_file_path = vid_files[0]
output_file_path = '../data/raw_data/temppp.mp4'

''.join([input_file_path, '  >>  ', output_file_path])

'../data/raw_data/video/594371324.545536.mp4  >>  ../data/raw_data/temppp.mp4'

## Get video info (ffprobe)

In [3]:
probe = ffmpeg.probe(input_file_path)
video_info = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

print(width, height, num_frames)
print(probe)

3840 2160 136
{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_time_base': '1001/48000', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 3840, 'height': 2160, 'coded_width': 3840, 'coded_height': 2160, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuvj420p', 'level': 51, 'color_range': 'pc', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'r_frame_rate': '24000/1001', 'avg_frame_rate': '24000/1001', 'time_base': '1/24000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 136136, 'duration': '5.672333', 'bit_rate': '66806828', 'bits_per_raw_sample': '8', 'nb_frames': '136', 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired

## Generate thumbnail for video
![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/get_video_thumbnail.png)

In [ ]:
import ffmpeg

stream = ffmpeg.input(input_file_path)
stream = ffmpeg.hflip(stream)
stream = ffmpeg.output(stream, 'output.mp4')
ffmpeg.run(stream)

# (
#     ffmpeg
#     .input('input.mp4')
#     .hflip()
#     .output('output.mp4')
#     .run()
# )




In [ ]:
import ffmpeg

in_file = ffmpeg.input('input.mp4')
overlay_file = ffmpeg.input('overlay.png')
(
    ffmpeg
    .concat(
        in_file.trim(start_frame=10, end_frame=20),
        in_file.trim(start_frame=30, end_frame=40),
    )
    .overlay(overlay_file.hflip())
    .drawbox(50, 50, 120, 120, color='red', thickness=5)
    .output('out.mp4')
    .run()
)

In [ ]:
https://kkroening.github.io/ffmpeg-python/
https://github.com/kkroening/ffmpeg-python


# input = ffmpeg.input('in.mp4')
# audio = input.audio.filter("aecho", 0.8, 0.9, 1000, 0.3)
# video = input.video.hflip()
# out = ffmpeg.output(audio, video, 'out.mp4')

In [ ]:
## custom filters

stream = ffmpeg.input('dummy.mp4')
stream = ffmpeg.filter(stream, 'fps', fps=25, round='up')
stream = ffmpeg.output(stream, 'dummy2.mp4')
ffmpeg.run(stream)


In [ ]:
(
    ffmpeg
    .input('in.mp4')
    .output('out.mp4', **{'qscale:v': 3})
    .run()
)

In [ ]:
## Multiple Input

main = ffmpeg.input('main.mp4')
logo = ffmpeg.input('logo.png')
(
    ffmpeg
    .filter([main, logo], 'overlay', 10, 10)
    .output('out.mp4')
    .run()
)

In [ ]:
## Multiple Output
split = (
    ffmpeg
    .input('in.mp4')
    .filter_multi_output('split')  # or `.split()`
)
(
    ffmpeg
    .concat(split[0], split[1].reverse())
    .output('out.mp4')
    .run()
)

In [ ]:
## String Expression
(
    ffmpeg
    .input('in.mp4')
    .filter('crop', 'in_w-2*10', 'in_h-2*20')
    .input('out.mp4')
)


In [ ]:
f = ffmpeg.input(input_file_path, ss=time)
print(type(f))
print(f)
f = f.filter('scale', width, -1)
print(type(f))
print(f)
f = f.output(output_file_path, vframes=1)
print(type(f))
print(f)
ffmpeg.run(f)

In [ ]:
import sys

print("fatal error", file=sys.stderr)

print(str(sys.stderr))

In [ ]:
(
    ffmpeg
    .input(input_file_path, ss=time)
    .filter('scale', width, -1)
    .output(output_file_path, vframes=1)
    .run()
)

## Convert video to numpy array

![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/ffmpeg-numpy.png)

In [ ]:
out, _ = (
    ffmpeg
    .input(file_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

In [ ]:
out

## Read Single Video as JPEG through pipe
![as](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/read_frame_as_jpeg.png)

In [ ]:


out, _ = (
    ffmpeg
    .input(in_filename)
    .filter('select', 'gte(n,{})'.format(frame_num))
    .output('pipe:', vframes=1, format='image2', vcodec='mjpeg')
    .run(capture_stdout=True)
)

## Convert sound to raw PCM audio

![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/transcribe.png)

In [ ]:


out, _ = (ffmpeg
    .input(in_filename, **input_kwargs)
    .output('-', format='s16le', acodec='pcm_s16le', ac=1, ar='16k')
    .overwrite_output()
    .run(capture_stdout=True)
)


## Assemble video from sequence of frames
![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/glob.png)

In [ ]:


(
    ffmpeg
    .input('/path/to/jpegs/*.jpg', pattern_type='glob', framerate=25)
    .output('movie.mp4')
    .run()
)


In [ ]:
(
    ffmpeg
    .input('/path/to/jpegs/*.jpg', pattern_type='glob', framerate=25)
    .filter('deflicker', mode='pm', size=10)
    .filter('scale', size='hd1080', force_original_aspect_ratio='increase')
    .output('movie.mp4', crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
    .view(filename='filter_graph')
    .run()
)

## Audio/video pipeline
![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/av-pipeline.png)

In [ ]:
in1 = ffmpeg.input('in1.mp4')
in2 = ffmpeg.input('in2.mp4')
v1 = in1.video.hflip()
a1 = in1.audio
v2 = in2.video.filter('reverse').filter('hue', s=0)
a2 = in2.audio.filter('areverse').filter('aphaser')
joined = ffmpeg.concat(v1, a1, v2, a2, v=1, a=1).node
v3 = joined[0]
a3 = joined[1].filter('volume', 0.8)
out = ffmpeg.output(v3, a3, 'out.mp4')
out.run()

## Mono to stereo with offsets and video

In [ ]:
audio_left = (
    ffmpeg
    .input('audio-left.wav')
    .filter('atrim', start=5)
    .filter('asetpts', 'PTS-STARTPTS')
)

audio_right = (
    ffmpeg
    .input('audio-right.wav')
    .filter('atrim', start=10)
    .filter('asetpts', 'PTS-STARTPTS')
)

input_video = ffmpeg.input('input-video.mp4')

(
    ffmpeg
    .filter((audio_left, audio_right), 'join', inputs=2, channel_layout='stereo')
    .output(input_video.video, 'output-video.mp4', shortest=None, vcodec='copy')
    .overwrite_output()
    .run()
)

# Viewer in Jupyter

In [4]:
# pip3 install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
from matplotlib import pyplot as plt
import ffmpeg
import numpy as np

In [5]:

probe = ffmpeg.probe(input_file_path)
video_info = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

print(width, height, num_frames)
print(video_info)

3840 2160 136
{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_time_base': '1001/48000', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 3840, 'height': 2160, 'coded_width': 3840, 'coded_height': 2160, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuvj420p', 'level': 51, 'color_range': 'pc', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'r_frame_rate': '24000/1001', 'avg_frame_rate': '24000/1001', 'time_base': '1/24000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 136136, 'duration': '5.672333', 'bit_rate': '66806828', 'bits_per_raw_sample': '8', 'nb_frames': '136', 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual

In [6]:
out, err = (
    ffmpeg
    .input(input_file_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width,3])
)

In [7]:
@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=136), Output()), _dom_classes=('widget-inter…

# Jupyter Stream Editor

In [9]:
from io import BytesIO
from PIL import Image


def extract_frame(stream, frame_num):
    while isinstance(stream, ffmpeg.nodes.OutputStream):
        stream = stream.node.incoming_edges[0].upstream_node.stream()
    out, _ = (
        stream
        .filter_('select', 'gte(n,{})'.format(frame_num))
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', vframes=1)
        .run(capture_stdout=True, capture_stderr=True)
    )
    return np.frombuffer(out, np.uint8).reshape([height, width, 3])


def png_to_np(png_bytes):
    buffer = BytesIO(png_bytes)
    pil_image = Image.open(buffer)
    return np.array(pil_image)
    

def build_graph(
        enable_overlay, flip_overlay, enable_box, box_x, box_y,
        thickness, color):

    stream = ffmpeg.input(input_file_path)

    if enable_overlay:
        overlay = ffmpeg.input('overlay.png')
        if flip_overlay:
            overlay = overlay.hflip()
        stream = stream.overlay(overlay)

    if enable_box:
        stream = stream.drawbox(
            box_x, box_y, 120, 120, color=color, t=thickness)

    return stream.output('out.mp4')


def show_image(ax, stream, frame_num):
    try:
        image = extract_frame(stream, frame_num)
        ax.imshow(image)
        ax.axis('off')
    except ffmpeg.Error as e:
        print(e.stderr.decode())


def show_graph(ax, stream, detail):
    data = ffmpeg.view(stream, detail=detail, pipe=True)
    image = png_to_np(data)
    ax.imshow(image, aspect='equal', interpolation='hanning')
    ax.set_xlim(0, 1100)
    ax.axis('off')


@interact(
    frame_num=(0, num_frames),
    box_x=(0, 200),
    box_y=(0, 200),
    thickness=(1, 40),
    color=['red', 'green', 'magenta', 'blue'],
)
def f(
        enable_overlay=True,
        enable_box=True,
        flip_overlay=True,
        graph_detail=False,
        frame_num=0,
        box_x=50,
        box_y=50,
        thickness=5,
        color='red'):

    stream = build_graph(
        enable_overlay,
        flip_overlay,
        enable_box,
        box_x,
        box_y,
        thickness,
        color
    )

    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15,4))
    plt.tight_layout()
    show_image(ax0, stream, frame_num)
    show_graph(ax1, stream, graph_detail)


interactive(children=(Checkbox(value=True, description='enable_overlay'), Checkbox(value=True, description='en…

## Tensorflow Streaming

https://github.com/kkroening/ffmpeg-python/blob/master/examples/tensorflow_stream.py

![](https://raw.githubusercontent.com/kkroening/ffmpeg-python/master/examples/graphs/tensorflow-stream.png)

- Decode input video with ffmpeg
- Process video with tensorflow using "deep dream" example
- Encode output video with ffmpeg

In [ ]:
process1 = (
    ffmpeg
    .input(in_filename)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24', vframes=8)
    .run_async(pipe_stdout=True)
)

process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
    .output(out_filename, pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

while True:
    in_bytes = process1.stdout.read(width * height * 3)
    if not in_bytes:
        break
    in_frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([height, width, 3])
    )

    # See examples/tensorflow_stream.py:
    out_frame = deep_dream.process_frame(in_frame)

    process2.stdin.write(
        out_frame
        .astype(np.uint8)
        .tobytes()
    )

process2.stdin.close()
process1.wait()
process2.wait()

## FaceTime webcam input (OS X)


In [ ]:
(
    ffmpeg
    .input('FaceTime', format='avfoundation', pix_fmt='uyvy422', framerate=30)
    .output('out.mp4', pix_fmt='yuv420p', vframes=100)
    .run()
)

## Stream from RTSP server to TCP socket

In [ ]:
packet_size = 4096

process = (
    ffmpeg
    .input('rtsp://%s:8554/default')
    .output('-', format='h264')
    .run_async(pipe_stdout=True)
)

while process.poll() is None:
    packet = process.stdout.read(packet_size)
    try:
        tcp_socket.send(packet)
    except socket.error:
        process.stdout.close()
        process.wait()
        break